# DataProcessing Notebook - AAAG

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
pd.set_option('display.max_columns', None)
df_train=pd.read_csv('Version_5_df.csv')

In [2]:
df_train.sample(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition,SalePrice
116,117,20,RL,71.0,11616,Pave,No Alley,Reg,Lvl,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1962,1962,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,116.0,3,3,CBlock,3,3,1,LwQ,1092,GasA,3,Y,SBrkr,1092,0,1092,0,1,1,0,3,3,6,Typ,1,1,Attchd,1962.000000,Unf,1,288,3,3,Y,0,1,1,0,0,0,No Fence,No Shed,9,2009,WD,Normal,11.842236
942,943,90,RL,42.0,7711,Pave,No Alley,IR1,Lvl,Inside,Gtl,Edwards,Norm,Norm,Duplex,1Story,4,3,1977,1977,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,PConc,4,3,4,GLQ,1440,GasA,3,Y,SBrkr,1440,0,1440,2,0,2,0,4,3,8,Typ,0,0,No Garage,1978.506164,No Garage,0,0,0,0,N,1,0,0,0,0,0,No Fence,No Shed,8,2007,Oth,Abnorml,11.918397
956,957,160,RM,24.0,1300,Pave,No Alley,Reg,Lvl,Inside,Gtl,Blueste,Norm,Norm,TwnhsE,2Story,6,6,1980,1980,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,CBlock,4,3,1,ALQ,561,GasA,3,Y,SBrkr,561,668,1229,0,0,1,1,2,3,5,Typ,1,3,Attchd,1980.000000,Fin,2,462,3,3,Y,1,0,0,0,0,0,GdPrv,No Shed,5,2009,WD,Normal,11.728045
1190,1191,190,RL,73.0,32463,Pave,No Alley,Reg,Low,Inside,Mod,Mitchel,Norm,Norm,2fmCon,1Story,4,4,1961,1975,Gable,CompShg,MetalSd,MetalSd,Stone,149.0,3,4,CBlock,3,3,3,BLQ,1249,GasA,5,Y,SBrkr,1622,0,1622,1,0,1,0,3,3,7,Typ,1,3,2Types,1975.000000,Fin,4,1356,3,3,Y,1,0,0,0,0,0,No Fence,No Shed,3,2007,WD,Normal,12.031725
394,395,50,RL,60.0,10134,Pave,No Alley,Reg,Lvl,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,5,6,1940,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,3,CBlock,3,3,1,Unf,735,GasA,4,Y,FuseA,735,299,1034,0,0,1,0,2,3,5,Typ,0,0,Detchd,1940.000000,Unf,1,240,3,3,Y,0,1,0,0,0,0,No Fence,No Shed,7,2007,WD,Normal,11.599112
610,611,60,RL,70.0,11050,Pave,No Alley,Reg,Lvl,Inside,Gtl,CollgCr,PosN,Norm,1Fam,2Story,9,5,2000,2000,Hip,CompShg,VinylSd,VinylSd,BrkFace,204.0,4,3,PConc,5,3,2,GLQ,1440,GasA,5,Y,SBrkr,1476,677,2153,1,0,2,1,3,5,8,Typ,2,5,Attchd,2000.000000,Fin,3,736,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,5,2009,WD,Normal,12.653962
565,566,70,RL,66.0,6858,Pave,No Alley,Reg,Bnk,Corner,Gtl,SWISU,Norm,Norm,1Fam,2Story,6,4,1915,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,3,3,PConc,4,3,1,Unf,806,GasA,3,N,FuseF,841,806,1647,1,0,1,1,4,2,6,Typ,0,0,Detchd,1920.000000,Unf,1,216,3,3,Y,0,1,1,0,0,0,No Fence,No Shed,5,2010,WD,Normal,11.759793
314,315,70,RM,60.0,9600,Pave,Grvl,Reg,Lvl,Inside,Gtl,OldTown,Norm,Norm,1Fam,2Story,7,7,1925,1990,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,3,3,BrkTil,3,4,1,LwQ,728,GasA,5,Y,SBrkr,832,809,1641,0,1,1,1,3,5,6,Typ,1,4,Detchd,1925.000000,Unf,2,546,2,3,Y,0,0,1,0,0,0,No Fence,No Shed,8,2006,WD,Normal,12.089544
1121,1122,20,RL,84.0,10084,Pave,No Alley,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4,3,PConc,4,3,3,GLQ,1552,GasA,5,Y,SBrkr,1552,0,1552,0,0,2,0,3,4,7,Typ,0,0,Attchd,2005.000000,RFn,3,782,3,3,Y,1,1,0,0,0,0,No Fence,No Shed,7,2006,New,Partial,12.268583
1099,1100,20,RL,82.0,11880,Pave,No Alley,IR1,Lvl,Inside,Gtl,NWAmes,RRAn,Norm,1Fam,1Story,7,5,1978,1978,Gable,CompShg,Plywood,Plywood,BrkFace,206.0,3,3,CBlock,4,3,1,ALQ,1271,GasA,3,Y,SBrkr,1601,0,1601,0,0,2,0,3,3,7,Typ,1,3,Attchd,1978.000000,RFn,2,478,3,3,Y,0,0,0,0,0,0,No Fence,No Shed,4,2009,COD,Abnorml,11.964007


In [3]:
# based on above graphs categorize into the following: 
nominal_var=['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','BsmtFinType2','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass']
ordinal_var=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath',\
             'HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'MoSold','YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond','PoolQC']

cont_var=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF',\
          'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']

var_dict={'nominal_var':nominal_var,'ordinal_var':ordinal_var,'cont_var':cont_var}

In [4]:
def feature_standardize(data,scaleType='standardize'):
    if scaleType not in ['standardize', 'normalize']: 
        raise ValueError('%s is not a valid choice' %(scaleType))
    mean_value=np.mean(data)
    standard_dev=np.std(data)
    min_value=np.min(data)
    max_value=np.min(data)
    if scaleType == 'standardize':
        return((data-mean_value)/standard_dev) 
    elif scaleType == 'normalize':
        return((data-min_value)/(max_value-min_value))

In [5]:
def dummify_column(dataframe,column_name):
    '''
    ### NOTE!!! ### vector operation not working yet!!!! ###
    - dataframe takes the entire dataframe you are working on
    - column_name takes a list of strings, where the strings are the column names
    '''
    for feature in column_name:
        dummified_feature = pd.get_dummies(dataframe.loc[:,feature], prefix=feature, prefix_sep='__',drop_first=True)
        dummified_df = pd.concat([dataframe.drop(feature,axis=1),dummified_feature],axis=1,sort='False')
    return dummified_df

In [6]:
def delete_outliers(dataframe,column_name,threshold=5):
    '''
    - Accepts a full dataframe
    - Specify column_name to indicate which columns to check for outliers
    - Column name accepts list of strings, where the strings are the column names
    - threshold is the number of deviations that will be used for removing outliers
    '''
    for feature in column_name:
        standard_dev=np.std(dataframe.loc[:,feature])
        dataframe=dataframe.drop(dataframe[dataframe.loc[:,feature]>standard_dev*threshold].index,axis=0)
    return dataframe


In [7]:
# Check to see if nominal_var is contained in df_train
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass']
i=0
for feature in nominal_var_processed:
    if not df_train.loc[:,feature].empty:
        i+=1
np.array(nominal_var_processed).size==i

True

In [8]:
# dummify the nominal categorical columns
df_train_copy=df_train.copy()
for feature in nominal_var_processed:
    df_train_copy=dummify_column(df_train_copy,[feature])
dummified_df=df_train_copy
print(dummified_df.shape)
print(dummified_df.isnull().sum().sum())
dummified_df.head(10)

(1460, 225)
0


,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,SalePrice,MSZoning__FV,MSZoning__RH,MSZoning__RL,MSZoning__RM,Street__Pave,Alley__No Alley,Alley__Pave,LotShape__IR2,LotShape__IR3,LotShape__Reg,LandContour__HLS,LandContour__Low,LandContour__Lvl,LotConfig__CulDSac,LotConfig__FR2,LotConfig__FR3,LotConfig__Inside,LandSlope__Mod,LandSlope__Sev,Neighborhood__Blueste,Neighborhood__BrDale,Neighborhood__BrkSide,Neighborhood__ClearCr,Neighborhood__CollgCr,Neighborhood__Crawfor,Neighborhood__Edwards,Neighborhood__Gilbert,Neighborhood__IDOTRR,Neighborhood__MeadowV,Neighborhood__Mitchel,Neighborhood__NAmes,Neighborhood__NPkVill,Neighborhood__NWAmes,Neighborhood__NoRidge,Neighborhood__NridgHt,Neighborhood__OldTown,Neighborhood__SWISU,Neighborhood__Sawyer,Neighborhood__SawyerW,Neighborhood__Somerst,Neighborhood__StoneBr,Neighborhood__Timber,Neighborhood__Veenker,Condition1__Feedr,Condition1__Norm,Condition1__PosA,Condition1__PosN,Condition1__RRAe,Condition1__RRAn,Condition1__RRNe,Condition1__RRNn,Condition2__Feedr,Condition2__Norm,Condition2__PosA,Condition2__PosN,Condition2__RRAe,Condition2__RRAn,Condition2__RRNn,BldgType__2fmCon,BldgType__Duplex,BldgType__Twnhs,BldgType__TwnhsE,HouseStyle__1.5Unf,HouseStyle__1Story,HouseStyle__2.5Fin,HouseStyle__2.5Unf,HouseStyle__2Story,HouseStyle__SFoyer,HouseStyle__SLvl,RoofStyle__Gable,RoofStyle__Gambrel,RoofStyle__Hip,RoofStyle__Mansard,RoofStyle__Shed,RoofMatl__CompShg,RoofMatl__Membran,RoofMatl__Metal,RoofMatl__Roll,RoofMatl__Tar&Grv,RoofMatl__WdShake,RoofMatl__WdShngl,Exterior1st__AsphShn,Exterior1st__BrkComm,Exterior1st__BrkFace,Exterior1st__CBlock,Exterior1st__CemntBd,Exterior1st__HdBoard,Exterior1st__ImStucc,Exterior1st__MetalSd,Exterior1st__Plywood,Exterior1st__Stone,Exterior1st__Stucco,Exterior1st__VinylSd,Exterior1st__Wd Sdng,Exterior1st__WdShing,Exterior2nd__AsphShn,Exterior2nd__Brk Cmn,Exterior2nd__BrkFace,Exterior2nd__CBlock,Exterior2nd__CmentBd,Exterior2nd__HdBoard,Exterior2nd__ImStucc,Exterior2nd__MetalSd,Exterior2nd__Other,Exterior2nd__Plywood,Exterior2nd__Stone,Exterior2nd__Stucco,Exterior2nd__VinylSd,Exterior2nd__Wd Sdng,Exterior2nd__Wd Shng,MasVnrType__BrkFace,MasVnrType__No Masonry Veneer,MasVnrType__None,MasVnrType__Stone,Foundation__CBlock,Foundation__PConc,Foundation__Slab,Foundation__Stone,Foundation__Wood,BsmtFinType1__BLQ,BsmtFinType1__GLQ,BsmtFinType1__LwQ,BsmtFinType1__No Basement,BsmtFinType1__Rec,BsmtFinType1__Unf,Heating__GasA,Heating__GasW,Heating__Grav,Heating__OthW,Heating__Wall,CentralAir__Y,Electrical__FuseF,Electrical__FuseP,Electrical__Mix,Electrical__SBrkr,Functional__Maj2,Functional__Min1,Functional__Min2,Functional__Mod,Functional__Sev,Functional__Typ,GarageType__Attchd,GarageType__Basment,GarageType__BuiltIn,GarageType__CarPort,GarageType__Detchd,GarageType__No Garage,GarageFinish__No Garage,GarageFinish__RFn,GarageFinish__Unf,PavedDrive__P,PavedDrive__Y,Fence__GdWo,Fence__MnPrv,Fence__MnWw,Fence__No Fence,MiscFeature__Shed,SaleType__CWD,SaleType__Con,SaleType__ConLD,SaleType__ConLI,SaleType__ConLw,SaleType__New,SaleType__Oth,SaleType__WD,SaleCondition__AdjLand,SaleCondition__Alloca,SaleCondition__Family,SaleCondition__Normal,SaleCondition__Partial,MSSubClass__30,MSSubClass__40,MSSubClass__45,MSSubClass__50,MSSubClass__60,MSSubClass__70,MSSubClass__75,MSSubClass__80,MSSubClass__85,MSSubClass__90,MSSubClass__120,MSSubClass__160,MSSubClass__180,MSSubClass__190
0,1,65.0,8450,7,5,2003,2003,196.0,4,3,4,3,1,856,5,856,854,1710,1,0,2,1,3,4,8,0,0,2003.0,2,548,3,3,0,1,0,0,0,0,2,2008,12.247699,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0

In [9]:
# For all continuous features, remove outliers where outlier >= 5 std
dummified_df_copy=dummified_df.copy()
for feature in ['TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageArea']:
    dummified_df_copy=delete_outliers(dummified_df_copy,[feature],threshold=5)
baseline_df=dummified_df_copy 
baseline_df.shape

(1290, 225)

In [10]:
# Check if ordinal variables are still there:
ordinal_var_processed=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath',\
             'HalfBath','BedroomAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'MoSold','YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond']

cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtUnfSF','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','GrLivArea','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF',\
          'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea']
i=0
for feature in ordinal_var_processed:
    if not baseline_df.loc[:,feature].empty:
        i+=1
np.array(ordinal_var_processed).size==i

True

In [11]:
# Check if continuous variables are still there:
cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','TotalBsmtSF',\
          '1stFlrSF','2ndFlrSF','GrLivArea','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF',\
          'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea']
i=0
for feature in cont_var_processed:
    if not baseline_df.loc[:,feature].empty:
        i+=1
np.array(cont_var_processed).size==i

True

In [12]:
baseline_df.to_csv('baseline_df.csv')